In [117]:
import json
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
log = math.log2
from sympy import *

In [118]:
channel_graph = nx.Graph()

In [198]:
channel_graph.add_edge("S","A",capacity=18)
channel_graph.add_edge("S","C",capacity=5)
channel_graph.add_edge("A","B",capacity=5)
channel_graph.add_edge("A","D",capacity=7)
#cross
channel_graph.add_edge("A","C",capacity=18)
channel_graph.add_edge("C","B",capacity=18)
channel_graph.add_edge("C","D",capacity=15)
channel_graph.add_edge("C","A",capacity=18)
#cross
channel_graph.add_edge("B","D",capacity=18)
channel_graph.add_edge("B","R",capacity=18)
channel_graph.add_edge("D","R",capacity=18)

In [199]:
def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def get_sid(src,dest):
    return "{}:{}".format(src,dest)

In [200]:
path_num = 0
channels = {}
for src,dest in channel_graph.edges():
    channels[get_sid(src,dest)]=[]
    channels[get_sid(dest,src)]=[]

print(channels)

used_variables = []
for path in nx.all_simple_paths(channel_graph,"S","R",cutoff=6):
    print(path_num, path)
    var = symbols("a{}".format(path_num))
    for src, dest in next_hop(path):
        sid = get_sid(src,dest)
        channels[sid].append(var)
    used_variables.append(var)
    path_num+=1
channels       
        

{'S:A': [], 'A:S': [], 'S:C': [], 'C:S': [], 'A:B': [], 'B:A': [], 'A:D': [], 'D:A': [], 'A:C': [], 'C:A': [], 'C:B': [], 'B:C': [], 'C:D': [], 'D:C': [], 'B:R': [], 'R:B': [], 'B:D': [], 'D:B': [], 'D:R': [], 'R:D': []}
0 ['S', 'A', 'B', 'C', 'D', 'R']
1 ['S', 'A', 'B', 'R']
2 ['S', 'A', 'B', 'D', 'R']
3 ['S', 'A', 'D', 'C', 'B', 'R']
4 ['S', 'A', 'D', 'R']
5 ['S', 'A', 'D', 'B', 'R']
6 ['S', 'A', 'C', 'B', 'R']
7 ['S', 'A', 'C', 'B', 'D', 'R']
8 ['S', 'A', 'C', 'D', 'R']
9 ['S', 'A', 'C', 'D', 'B', 'R']
10 ['S', 'C', 'B', 'A', 'D', 'R']
11 ['S', 'C', 'B', 'R']
12 ['S', 'C', 'B', 'D', 'R']
13 ['S', 'C', 'D', 'A', 'B', 'R']
14 ['S', 'C', 'D', 'R']
15 ['S', 'C', 'D', 'B', 'R']
16 ['S', 'C', 'A', 'B', 'R']
17 ['S', 'C', 'A', 'B', 'D', 'R']
18 ['S', 'C', 'A', 'D', 'R']
19 ['S', 'C', 'A', 'D', 'B', 'R']


{'A:B': [a0, a1, a2, a13, a16, a17],
 'A:C': [a6, a7, a8, a9],
 'A:D': [a3, a4, a5, a10, a18, a19],
 'A:S': [],
 'B:A': [a10],
 'B:C': [a0],
 'B:D': [a2, a7, a12, a17],
 'B:R': [a1, a3, a5, a6, a9, a11, a13, a15, a16, a19],
 'C:A': [a16, a17, a18, a19],
 'C:B': [a3, a6, a7, a10, a11, a12],
 'C:D': [a0, a8, a9, a13, a14, a15],
 'C:S': [],
 'D:A': [a13],
 'D:B': [a5, a9, a15, a19],
 'D:C': [a3],
 'D:R': [a0, a2, a4, a7, a8, a10, a12, a14, a17, a18],
 'R:B': [],
 'R:D': [],
 'S:A': [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9],
 'S:C': [a10, a11, a12, a13, a14, a15, a16, a17, a18, a19]}

In [201]:
def channel_probability(capacity,expression):
    return (capacity+1-expression)/(capacity+1)

def create_target_function():
    res = 1
    for sid,variables in channels.items():
        expression=0
        for variable in variables:
            expression+=variable
        src,dest = sid.split(":")
        eq = channel_probability(channel_graph[src][dest]["capacity"],expression)
        #print(eq)
        res*=eq
    return res

f = create_target_function()

In [202]:
print(f)

1.0*(-a0/19 + 1)*(-a10/6 + 1)*(-a13/8 + 1)*(-a3/16 + 1)*(-a12/19 - a17/19 - a2/19 - a7/19 + 1)*(-a15/19 - a19/19 - a5/19 - a9/19 + 1)*(-a16/19 - a17/19 - a18/19 - a19/19 + 1)*(-a6/19 - a7/19 - a8/19 - a9/19 + 1)*(-a0/6 - a1/6 - a13/6 - a16/6 - a17/6 - a2/6 + 1)*(-a0/16 - a13/16 - a14/16 - a15/16 - a8/16 - a9/16 + 1)*(-a10/8 - a18/8 - a19/8 - a3/8 - a4/8 - a5/8 + 1)*(-a10/19 - a11/19 - a12/19 - a3/19 - a6/19 - a7/19 + 1)*(-0.0526315789473684*a0 - 0.0526315789473684*a1 - 0.0526315789473684*a2 - 0.0526315789473684*a3 - 0.0526315789473684*a4 - 0.0526315789473684*a5 - 0.0526315789473684*a6 - 0.0526315789473684*a7 - 0.0526315789473684*a8 - 0.0526315789473684*a9 + 1.0)*(-a0/19 - a10/19 - a12/19 - a14/19 - a17/19 - a18/19 - a2/19 - a4/19 - a7/19 - a8/19 + 1)*(-a1/19 - a11/19 - a13/19 - a15/19 - a16/19 - a19/19 - a3/19 - a5/19 - a6/19 - a9/19 + 1)*(-a10/6 - a11/6 - a12/6 - a13/6 - a14/6 - a15/6 - a16/6 - a17/6 - a18/6 - a19/6 + 1)


In [203]:
cache = set()
def gen(x,a):
    s = sum(x)
    if s == a:
        string = ",".join(str(y) for y in x)
        if string not in cache:
            cache.add(string)
            yield(x)
    if s < a:
        for i in range(len(x)):
            x[i]+=1
            yield from gen(x,a)
            x[i]-=1


In [204]:
x = [0 for var in used_variables]
#x = [0,0,0]
a = 3

print("start")
m = 0
b = None
for res in gen(x,a):
    subs=dict(zip(used_variables,res))
    e = (f).evalf(subs = subs)
    #print(e)
    if e>m:
        m=e
        b = res.copy()
        print(m,res)

print(m,b)

start
0.242600962239394 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.279417745413249 [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.289528255938070 [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.323856733245857 [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.328456424803693 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.369817604223418 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.393643388249016 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.416387228458959 [0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.422648690541049 [0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.446129173348885 [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.467138026387475 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.472699193368278 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.480484827141403 [0, 0, 0, 0, 2, 0, 1, 0, 0, 

In [234]:
num_vars = len(used_variables)
p = [a/num_vars for var in used_variables]
subs=dict(zip(used_variables,p))
best_prob =  (f).evalf(subs = subs)
print(sum(p),best_prob,p)

d = p[0]/100
for k in range(100):
    print("round: ",k)
    for i in range(num_vars):
        go= True
        while p[i]-d>0 and go:    
            p[i]-=d
            for j in range(i+1,num_vars):
                p[j]+=d/(num_vars-i)
            subs=dict(zip(used_variables,p))
            prob = (f).evalf(subs = subs)
            if prob < best_prob:
                p[i]+=d
                for j in range(i+1,num_vars):
                    p[j]-=d/(num_vars-i)
                go=False
            else:
                best_prob=prob
                print(sum(p),prob,p)


2.999999999999999 0.328694202256670 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]
round:  0
2.9999250000000006 0.328720108553688 [0.1485, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075, 0.150075]
2.99985 0.328745983832380 [0.147, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998, 0.15014999999999998]
2.999774999999999 0.328771828089601 [0.1455, 0.15022499999999997, 0.15022499999999997, 0.15022499999999997, 0.15022499999999997, 0.15022499999999997, 0.150224999999

2.9972250000000003 0.330395804249252 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1313249999999999, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005, 0.15277500000000005]
2.9970750000000006 0.330504895138357 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.12982499999999988, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006, 0.15292500000000006]
2.996925000000001 0.330613984236631 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.150

2.9933250000000022 0.333231555222905 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.09232499999999985, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034, 0.15667500000000034]
2.993175000000003 0.333340595785298 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.09082499999999985, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035, 0.15682500000000035]
2.9930250000000034 0.333449634255696 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.15

2.9894250000000047 0.336065899849316 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.05332499999999982, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063, 0.16057500000000063]
2.989275000000005 0.336174881085753 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.051824999999999816, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065, 0.16072500000000065]
2.989125000000006 0.336283862273716 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.15

2.9855250000000066 0.338898634713377 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.014324999999999784, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093, 0.16447500000000093]
2.9853750000000074 0.339007551148357 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.1508249999999999, 0.012824999999999784, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094, 0.16462500000000094]
2.9852250000000082 0.339116464889823 [0.13349999999999998, 0.1508249999999999, 0.1508249999999999, 0

...

2.187877100840321 0.524287782382030 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.004601470588235141, 0.18280147058823484, 0.18280147058823484, 0.017801470588236255, 0.2063728991596643, 0.2063728991596643, 0.0173728991596657, 0.01812289915966395, 0.017122899159663925, 0.017122899159658096]
2.1877271008403207 0.524454642758349 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.003101470588235141, 0.18295147058823485, 0.18295147058823485, 0.017951470588236255, 0.20652289915966432, 0.20652289915966432, 0.017522899159665702, 0.01827289915966395, 0.017272899159663926, 0.017272899159658097]
2.18757710084032 0.524621500683077 [0.001499999999999879, 0.1574249999999

2.183730672268892 0.527370988368056 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.015894327731094286, 0.02226932773109254, 0.021269327731092514, 0.021269327731086685]
2.1833556722688923 0.527462763863353 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.014394327731094286, 0.02264432773109254, 0.021644327731092514, 0.021644327731086686]
2.1829806722688923 0.527554522814717 [0.001499999999999879, 0.157424999

2.174480672268892 0.530262205650276 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.009519327731092537, 0.030519327731092522, 0.030519327731086694]
2.1739806722688924 0.530441698544146 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.008019327731092537, 0.031019327731092523, 0.031019327731086694]
2.1734806722688917 0.530621194602291 [0.001499999999999879, 0.15742

2.1587306722688924 0.533918544456452 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.0005193277310925376, 0.00801932773109251, 0.046269327731086704]
2.1579806722688923 0.534070453288506 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.0005193277310925376, 0.0065193277310925096, 0.047019327731086705]
2.1572306722688923 0.534222382470151 [0.001499999999999879, 0.15

2.1249806722688924 0.542548208348396 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.0005193277310925376, 0.0005193277310925099, 0.02001932773108668]
2.1234806722688924 0.542943919895832 [0.001499999999999879, 0.15742499999999918, 0.15742499999999918, 0.0014249999999990546, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.16660147058823363, 0.00010147058823514092, 0.18325147058823488, 0.18325147058823488, 0.0002514705882362589, 0.209394327731093, 0.209394327731093, 0.0008943277310942883, 0.0005193277310925376, 0.0005193277310925099, 0.01851932773108668]
2.1219806722688923 0.543339843038009 [0.001499999999999879, 0.15

In [221]:
num_vars = len(used_variables)
p = [a/num_vars for var in used_variables]
#p = [0, 0, 0]
# Define potential changes
dp = [0.05* t for t in p]
# Calculate the error

best_err = 0

threshold = 0.001

print(p,dp)

while sum(dp) > threshold:
    for i in range(len(p)):
        other = False
        if p[i]+dp[i]>0:
            p[i] += dp[i]
            for j in range(len(p)):
                if p[j]-dp[i]/len(p) < 0:
                    other = True
                p[j]-=dp[i]/len(p)
            if not other:
                subs=dict(zip(used_variables,p))
                err =  (f).evalf(subs = subs)

                if err > best_err:  # There was some improvement
                    best_err = err
                    dp[i] *= 1.1
                    print(p,err,sum(p))
                else:                
                    other=True
        if p[i]+dp[i]<0 or other:  # There was no improvement
            x = 1
            if other:
                x=2
            p[i] -= x*dp[i]
            revert = False
            for j in range(len(p)):
                if p[j]+x*dp[i]/len(p)<0:
                    revert = True
                p[j]+=x*dp[i]/len(p)
            
            if revert:
                p[i] += dp[i]
                for j in range(len(p)):                
                    p[j]-=dp[i]/len(p)
                continue
            
            subs=dict(zip(used_variables,p))
            err = (f).evalf(subs = subs)

            if err > best_err:  # There was an improvement
                best_err = err
                dp[i] *= 1.05
                print(p,err,sum(p))
            else:  # There was no improvement
                p[i] += dp[i]
                # As there was no improvement, the step size in either
                # direction, the step size might simply be too big.
                dp[i] *= 0.95


[0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15] [0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075, 0.0075]
[0.15712500000000001, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625, 0.149625] 0.328617180893161 2.9999999999999987
[0.15675000000000003, 0.15675000000000003, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002, 0.14925000000000002] 0.328833658187611 3.0
[0.15637500000000004, 0.15

[0.14370000000000016, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.16020000000000018, 0.13695000000000015, 0.15195000000000017, 0.15195000000000017, 0.13695000000000015, 0.15195000000000017, 0.13695000000000018, 0.13695000000000018, 0.13695000000000018, 0.13695000000000018, 0.13695000000000018] 0.336511293138106 3.000000000000003
[0.14409375000000016, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.16059375000000017, 0.12946875000000013, 0.15234375000000017, 0.15234375000000017, 0.13734375000000015, 0.15234375000000017, 0.13734375000000018, 0.13734375000000018, 0.13734375000000018, 0.13734375000000018, 0.13734375000000018] 0.337161966539570 3.000000000000004
[0.14368125000000015, 0.16018125000000016, 0.16018125000000016, 0.16018125000000016, 0

[0.1325718750000001, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.16680937500000012, 0.15773437500000012, 0.12660937500000008, 0.15773437500000012, 0.15773437500000012, 0.1266093750000001, 0.15773437500000012, 0.14235937500000012, 0.12660937500000014, 0.1266093750000001, 0.12660937500000014, 0.1266093750000001] 0.342222074963310 3.0000000000000027
[0.1321181250000001, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.16635562500000012, 0.12615562500000008, 0.15728062500000012, 0.15728062500000012, 0.1261556250000001, 0.15728062500000012, 0.14190562500000012, 0.12615562500000013, 0.1261556250000001, 0.12615562500000013, 0.1261556250000001] 0.342548759610724 3.0000000000000013
[0.1325315625000001, 0.16676906250000012, 0.16676906250000012, 0.16676906250000012, 0.166769

[0.11899992187500001, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.17231554687500028, 0.10518210937500001, 0.17231554687500028, 0.17231554687500028, 0.10518210937500004, 0.17231554687500028, 0.15607429687500027, 0.10518210937500007, 0.11386429687500005, 0.11386429687500008, 0.11386429687500005] 0.351503918923177 3.000000000000004
[0.11943403125, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.1727496562500003, 0.10561621875, 0.1727496562500003, 0.1727496562500003, 0.10561621875000003, 0.1727496562500003, 0.1565084062500003, 0.10561621875000006, 0.10561621875000003, 0.11429840625000007, 0.11429840625000004] 0.352047860190042 3.0000000000000044
[0.119868140625, 0.17318376562500032, 0.17318376562500032, 0.17318376562500032, 0.17318376562500032, 0.1731837

[0.10555254374999994, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.17939932500000044, 0.09216884062499997, 0.17939932500000044, 0.17939932500000044, 0.092168840625, 0.17939932500000044, 0.15217732500000042, 0.10128513750000002, 0.1012851375, 0.10128513750000002, 0.1012851375] 0.358424497787564 3.000000000000005
[0.10502846249999993, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.17887524375000044, 0.09164475937499997, 0.17887524375000044, 0.17887524375000044, 0.091644759375, 0.17887524375000044, 0.1621348687500004, 0.10076105625000002, 0.10076105625, 0.10076105625000002, 0.10076105625] 0.358431697879676 3.0000000000000058
[0.10548427734374993, 0.17933105859375043, 0.17933105859375043, 0.17933105859375043, 0.17933105859375043, 0.17933105859375

...

[0.06627623683066389, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.19710158906894573, 0.06488274317050768, 0.19710158906894573, 0.19710158906894573, 0.06488274317050771, 0.19710158906894573, 0.1792082353189457, 0.06488274317050773, 0.06488274317050773, 0.06488274317050774, 0.06488274317050774] 0.377030974923318 3.0000000000000044
[0.06554546791816389, 0.21098619840644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.19637082015644572, 0.06415197425800769, 0.19637082015644572, 0.19637082015644572, 0.06415197425800771, 0.19637082015644572, 0.17847746640644568, 0.06415197425800773, 0.06415197425800773, 0.06415197425800774, 0.06415197425800774] 0.377499199636171 3.000000000000006
[0.06481469900566389, 0.2102554294939457, 0.2102554294939457, 0.1956400512439457, 0.19

[0.04894728412532208, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.20599659310051774, 0.04860911578215807, 0.20599659310051774, 0.20599659310051774, 0.0486091157821581, 0.20599659310051774, 0.1874389039755177, 0.04860911578215811, 0.04860911578215811, 0.048609115782158124, 0.048609115782158124] 0.386642152583449 3.0000000000000004
[0.04814343832157208, 0.22126966337176773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.20519274729676773, 0.04780526997840807, 0.20519274729676773, 0.20519274729676773, 0.047805269978408095, 0.20519274729676773, 0.1866350581717677, 0.04780526997840811, 0.04780526997840811, 0.04780526997840812, 0.04780526997840812] 0.387167531193097 3.000000000000001
[0.04733959251782208, 0.22046581756801772, 0.22046581756801772, 0.20438890149301772, 

[0.03027854483002566, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.21559377729148108, 0.03104846806970347, 0.21559377729148108, 0.21559377729148108, 0.031048468069703496, 0.21559377729148108, 0.19630531925398106, 0.031048468069703503, 0.0310484680697035, 0.03104846806970351, 0.031048468069703506] 0.397154040709069 3.0000000000000018
[0.02939431444590066, 0.2323941545898561, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.21470954690735608, 0.030164237685578472, 0.21470954690735608, 0.21470954690735608, 0.030164237685578493, 0.21470954690735608, 0.19542108886985607, 0.0301642376855785, 0.0301642376855785, 0.030164237685578507, 0.030164237685578507] 0.397744355818216 2.9999999999999996
[0.02851008406177566, 0.2315099242057311, 0.2315099242057311, 0.21382531652323108

[0.022314230612539815, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.22531407075649523, 0.011449192232378733, 0.22531407075649523, 0.22531407075649523, 0.011449192232378757, 0.22531407075649523, 0.2052217669152452, 0.011449192232378775, 0.011449192232378775, 0.011449192232378788, 0.011449192232378788] 0.408484910066804 3.000000000000001
[0.010155695719808193, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.22595399364558635, 0.01208911512146987, 0.22595399364558635, 0.22595399364558635, 0.012089115121469894, 0.22595399364558635, 0.20586168980433633, 0.012089115121469911, 0.012089115121469911, 0.012089115121469925, 0.012089115121469925] 0.408666401553813 2.999999999999999
[0.009183042297270693, 0.24443440867379887, 0.22498134022304886, 0.2249813

KeyboardInterrupt: 

In [ ]:
subs=dict(zip(variables[:-1],res[:-1]))
print((f).evalf(subs = subs))

In [81]:
print(x)

[]


In [105]:
for res in gen([0,0,0],3):
    print(res)

[3, 0, 0]
[2, 1, 0]
[2, 0, 1]
[1, 2, 0]
[1, 1, 1]
[1, 0, 2]
[0, 3, 0]
[0, 2, 1]
[0, 1, 2]
[0, 0, 3]


In [113]:
def create_and_solve_lagrange_problem(f,variables,amt):
    partial_derrivatives = [diff(f,var) for var in variables]
    l = symbols("l")
    non_linear_equations = [Eq(p,l) for p in partial_derrivatives]
    split = 0
    for var in variables:
        split=split+var
    variables.append(l)
    non_linear_equations.append(Eq(split,amt))
    start = [amt/(len(variables)-1) for var in variables[:-1]]
    start.append(1)
    print(non_linear_equations,split, start)
    print((f).evalf(subs=dict(zip(variables[:-1],start))))
    res = nsolve(non_linear_equations,variables,start, simplify=False)
    print(amt, res)
    subs=dict(zip(variables[:-1],res[:-1]))
    print((f).evalf(subs = subs))

In [62]:
print(used_variables)
create_and_solve_lagrange_problem(f,used_variables,16)

[a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11]
[Eq(0.0909090909090909*(a0/11 + 1)*(a2/11 + 1)*(a6/11 + 1)*(a8/11 + 1)*(a10/11 + a11/11 + 1)*(a4/11 + a5/11 + 1)*(a0/11 + a1/11 + a10/11 + a8/11 + 1)*(a0/11 + a5/11 + a8/11 + a9/11 + 1)*(a11/11 + a2/11 + a3/11 + a6/11 + 1)*(a2/11 + a4/11 + a6/11 + a7/11 + 1)*(0.0909090909090909*a0 + 0.0909090909090909*a1 + 0.0909090909090909*a2 + 0.0909090909090909*a3 + 0.0909090909090909*a4 + 0.0909090909090909*a5 + 1.0)*(a1/11 + a10/11 + a2/11 + a4/11 + a7/11 + a8/11 + 1)*(a10/11 + a11/11 + a6/11 + a7/11 + a8/11 + a9/11 + 1) + 0.0909090909090909*(a0/11 + 1)*(a2/11 + 1)*(a6/11 + 1)*(a8/11 + 1)*(a10/11 + a11/11 + 1)*(a4/11 + a5/11 + 1)*(a0/11 + a1/11 + a10/11 + a8/11 + 1)*(a0/11 + a5/11 + a8/11 + a9/11 + 1)*(a11/11 + a2/11 + a3/11 + a6/11 + 1)*(a2/11 + a4/11 + a6/11 + a7/11 + 1)*(a0/11 + a11/11 + a3/11 + a5/11 + a6/11 + a9/11 + 1)*(a1/11 + a10/11 + a2/11 + a4/11 + a7/11 + a8/11 + 1)*(a10/11 + a11/11 + a6/11 + a7/11 + a8/11 + a9/11 + 1) + 0.090909090909

ZeroDivisionError: matrix is numerically singular